In [ ]:

from egnn_pytorch import EGNN_Network
import torch
import torch.nn as nn
import numpy as np
import glob
import time
import gzip
import math
plosses=[]
class RadialBasisFunction(torch.nn.Module):
    def __init__(self, num_basis=128, cutoff=5.0):
        super().__init__()
        self.cutoff = cutoff
        self.register_buffer('mu', torch.linspace(0, cutoff, num_basis))
        self.gamma = 12 # You can tune this

    def forward(self, distances):
        distances = distances.unsqueeze(-1) # shape (N, N, 1)
        rbf = torch.exp(-self.gamma * (distances - self.mu)**2)
        return rbf # shape (N, N, num_basis)

def pairwise_distances(x):
    diff = x.unsqueeze(1) -x.unsqueeze(0)
    dist = torch.norm(diff,dim=-1)
    return dist
# Example of aggregation function
def aggregate_rbf_features(rbf_features):
# Simple global mean aggregation
    global_features = rbf_features.mean(dim=(0,1))
    return global_features

# After EGNN outputs embeddings (h_i, x_i):
#distances = pairwise_distances(out1[1])
#rbf = RadialBasisFunction()(distances)
class TinyRegressor(nn.Module):
    def __init__(self, in_channels=3,lin_channels=dim):
        super().__init__()
        #self.encoding = A
        #self.mha = mha
        self.conv1 = nn.Conv1d(in_channels, 15, 1, padding = 0)
        self.conv2 = nn.Conv1d(15, 3, 3, padding=1)
        #self.conv3 = nn.Conv1d(3, 2, 2, padding=0)
        self.conv4 = nn.Conv1d(3, 1, 1, padding=0)
        self.out  = nn.Linear(1, 1)
        self.conv5 = nn.Conv1d(1, 1, 3, padding = (3 - 1) // 2)
        #self.conv6 = nn.Conv1d(1, 1, 1, padding = (1 - 1) // 2)

        #self.softmax=nn.Softmax()
            #nn.ReLU(),
        #nn.Conv1d(3,1,1, padding=1),
            #nn.ReLU()
        #)
        self.relu=nn.PReLU()#Softmax()#ReLU()
        #self.pool = nn.AdaptiveMaxPool2d((1, 1))
        #self.out1  = nn.Linear(6, 1)
        
        

    def forward(self, x):
        #x=self.mha(self.encoding(x))
        x=self.relu(x)
        x = self.conv1(x)
        x=self.relu(x)
        x = self.conv2(x)
        x=self.relu(x)
        x=self.out(x.T)
        
        #x = self.conv3(x)
        #x=self.relu(x)
        
        x = self.conv4(x)
        #x = self.conv4(x)
        #x = self.softmax(x)
        #x=self.out1(x)
        #x = self.pool(x).flatten(1)   # (B, 32)
        #x=self.out(x)#self.relu(x)
        return x#self.relu(x)

       # (B, 1)

# ---------------------------------------------------------------
# 1) Helper to load one sample and keep it in the graph
# ---------------------------------------------------------------
def load_example(path, device):
    a = np.load(path, allow_pickle=True)
    zs, xs, ys = a["z"], a["pos"], a["pks"]
    # convert to torch, keep autograd graph
    zs_t = torch.from_numpy(zs.astype(np.int32)).to(device)       # for embedding lookups
    xs_t = torch.from_numpy(xs).to(device).float()                # your float features
    ys_t = torch.from_numpy(ys).to(device).float()                # your targets
    return zs_t, xs_t, ys_t

# ---------------------------------------------------------------
# 2) Build your models, criterion, device
# ---------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#net   = YourBackboneNet(...).to(device)
#mha   = nn.MultiheadAttention(embed_dim=..., num_heads=...).to(device)
#model = YourScalarHead(...).to(device)
def egnn(dim=6):#,learning_rate=.001):

    net=EGNN_Network(dim=dim,
    depth=1,
    num_positions=500, #no
    num_tokens=100,
    num_nearest_neighbors=1,
    #coor_weights_clamp_value = 2,
    norm_coors=True)

    #optimizer= torch.optim.Adam(list(net.parameters()) + list(model.parameters()), lr=learning_rate)
    
    #criterion = nn.L1Loss()



    return net#, optimizer, criterion
dim=10
criterion = nn.L1Loss()
model = TinyRegressor(in_channels=128+dim,lin_channels=dim).to(device)
net=egnn(dim=dim).to(device)
A = PositionalEncoding(dim)
mha = SimpleMultiheadAttention(dim,dim).to(device) #6 is the dim
torch.manual_seed(0)
vlosses=[]

# ---------------------------------------------------------------
# 3) Single optimizer with two param‐groups
# ---------------------------------------------------------------
optimizer = torch.optim.AdamW([
    {"params": net.parameters(),   "lr": 1e-4},
    {"params": mha.parameters(),   "lr": 1e-2},
    {"params": model.parameters(), "lr": 1e-3},
],weight_decay=1e-5)

# ---------------------------------------------------------------
# 3) Get file list
# ---------------------------------------------------------------
paths = sorted(glob.glob("/home/jrhoernschemeyer/Desktop/data_prep/inputs/*.npz"))[:100]
tpaths=sorted(glob.glob("/home/jrhoernschemeyer/Desktop/data_prep/inputs/*.npz"))[100:200]
epoch_losses = []



# ---------------------------------------------------------------
# 4) Training loop
# ---------------------------------------------------------------
for epoch in range(30):
    print(f"Epoch {epoch}")
    net.train()
    mha.train()
    model.train()
    plosses.append(epoch_losses)
    vlosses.append(vepoch_losses)
    epoch_losses = []
    vepoch_losses = []
    #vlosses=[]

    # for brevity, we’ll do just the first 2 files as before
    for path in list(np.array(paths)[np.random.permutation(len(paths))]):#np.shuffle(paths):
        # load the raw arrays
        a  = np.load(path, allow_pickle=True)
        zs = a["z"]       # array of length-N residues, each zs[i] is a (Li,3) array
        xs = a["pos"]     # (N,?) array of floats
        ys = a["pks"]     # (N,) array of targets

        optimizer.zero_grad()
        outs = []

        # now iterate residue-by-residue, converting each to a proper tensor
        for z_i, x_i, y_i in zip(zs, xs, ys):
            # convert this one example
            # — for z: a variable-length coordinate array
            z_t = torch.from_numpy(z_i[0].astype(np.int32)) \
                       .unsqueeze(0) \
                       .to(device)            # [1, Li, 3] or however your net expects it

            # — for x: your feature array, assumed float
            x_t = torch.from_numpy(x_i).unsqueeze(0).float().to(device)  # [1, ?]

            # forward pass
            out1, _         = net(z_t, x_t)           # adapt to your signature
            attn_out,__     = mha(A(out1[0]))
            distances = pairwise_distances(_)
            rbf = RadialBasisFunction()(distances)
            global_rbf_features = aggregate_rbf_features(rbf)
           

            
            #out             = attn_out#out1#torch.matmul(__,attn_out)
            global_node_features    = attn_out.max(dim=0)[0]   # [features]#
            #global_node_features = out
            # Concatenate node embeddings and RBF features
            global_embedding = torch.hstack([global_node_features, global_rbf_features]).unsqueeze(2)

            # Predict energy scalar
            energy_pred = model(global_embedding.permute(2,1,0))
            #attn_out,_ =       mha(A(out1[0]))
            #embedding       = global_node_feats # [features, 1]
            #energy_pred     = model(embedding.permute(0,2,1))#.unsqueeze(0))           # [1,1] or similar
            outs.append(torch.mean(energy_pred))#.squeeze())

        # stack and compute loss
        preds = torch.hstack(outs)  
        #preds=model.conv5(mha(A(preds.unsqueeze(1)))[0].permute(1,2,0)) 
        #preds=model.conv6(preds)[0]
        preds = model.conv5(preds.unsqueeze(0).unsqueeze(1))[0][0]          # [N]
        target = torch.from_numpy(ys).float().to(device)

        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

        #print(f"Epoch {epoch}")
    
############################
    net.eval()
    mha.eval()
    model.eval()
    vlosses.append(vepoch_losses)
        # for brevity, we’ll do just the first 2 files as before
    for path in tpaths:#np.shuffle(paths):
        # load the raw arrays
        a  = np.load(path, allow_pickle=True)
        zs = a["z"]       # array of length-N residues, each zs[i] is a (Li,3) array
        xs = a["pos"]     # (N,?) array of floats
        ys = a["pks"]     # (N,) array of targets

        #optimizer.zero_grad()
        outs = []

        # now iterate residue-by-residue, converting each to a proper tensor
        for z_i, x_i, y_i in zip(zs, xs, ys):
            # convert this one example
            # — for z: a variable-length coordinate array
            z_t = torch.from_numpy(z_i[0].astype(np.int32)) \
                    .unsqueeze(0) \
                    .to(device)            # [1, Li, 3] or however your net expects it

            # — for x: your feature array, assumed float
            x_t = torch.from_numpy(x_i).unsqueeze(0).float().to(device)  # [1, ?]

            # forward pass
            out1, _         = net(z_t, x_t)           # adapt to your signature
            attn_out,__     = mha(A(out1[0]))
            distances = pairwise_distances(_)
            rbf = RadialBasisFunction()(distances)
            global_rbf_features = aggregate_rbf_features(rbf)
        

            
            out             = attn_out#out1#torch.matmul(__,attn_out)
            global_node_features    = out.max(dim=0)[0]   # [features]#
            #global_node_features = out
            # Concatenate node embeddings and RBF features
            global_embedding = torch.hstack([global_node_features, global_rbf_features]).unsqueeze(2)

            # Predict energy scalar
            energy_pred = model(global_embedding.permute(2,1,0))
            #attn_out,_ =       mha(A(out1[0]))
            #embedding       = global_node_feats # [features, 1]
            #energy_pred     = model(embedding.permute(0,2,1))#.unsqueeze(0))           # [1,1] or similar
            outs.append(torch.mean(energy_pred))#.squeeze())

        # stack and compute loss
        preds = torch.hstack(outs)  
        #preds=model.conv5(mha(A(preds.unsqueeze(1)))[0].permute(1,2,0)) 
        #preds=model.conv5(preds.unsqueeze(1)))[0].permute(1,2,0
        #preds=model.conv6(preds)[0]
        preds = model.conv5(preds.unsqueeze(0).unsqueeze(1))[0][0]          # [N]
        target = torch.from_numpy(ys).float().to(device)
        
        loss = criterion(preds, target)
        #loss.backward()
        #optimizer.step()

        vepoch_losses.append(loss.item())




    avg = sum(epoch_losses) / len(epoch_losses)
    print(f" → avg loss: {avg:.4f}\n")

    avg = sum(vepoch_losses) / len(vepoch_losses)
    print(f" → avg loss: {avg:.4f}\n")


plt.plot(np.array([plosses[1:]]).flatten())
plt.show()
plt.plot(np.array([vlosses[1:]]).flatten())

plt.title(f"now conv1ding the preds. dim4. 2gamma.1e4 . mean egnn {egnn} \n {model}")
plt.show()
